In [14]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd
import numpy as np

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [15]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch_prep_policy.gdb", 'policy_override.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])


In [16]:
# read in data
new_polygons = r".\2019\policy_override.gdb\policy_override\policy_override_polygons2"
policy_polys = pd.DataFrame.spatial.from_featureclass(new_polygons)
policy_polys.head()

,OBJECTID,AreaName,AreaType,Phase,CityPrimary,CityOther,CatchmentPop,NonResFAR,DwellingPerAcre,IndicatorLU,MinRoadway,Transit,Walkable,Stories,Notes,id,SHAPE
0,1,West Layton Town Center,City Center,2,Layton,None,"25,000",0.5 to 1.5,10 to 50,"Grocery store, + Neighborhood Center uses",Intersection of major collector with an arterial,Intersection of frequent bus lines --or-- serv...,Yes,2 to 5,"Amendment 4: Boundary Adjustment, Type Change",1,"{""rings"": [[[415395.8512000004, 4546928.454], ..."
1,2,Layton Industrial Center,Industrial District,2,Layton,None,None,N/A,N/A,"Light and heavy industry, warehousing, inciden...",Freight oriented,Bus service,No,None,Amendment 4: Boundary Adjustment,2,"{""rings"": [[[416374.0889999997, 4547760.0077],..."
2,3,Layton Midtown,Urban Center,1,Layton,Clearfield,"100,000",0.75 to 4.0,20 to 100,"Significant commerce. Regional ""destination"" r...",Intersection of major arterials and/or freeway...,Junction points of the high capacity transit n...,Yes,4 to 10,Amendment 4: Boundary Adjustment,3,"{""rings"": [[[417032.33100000024, 4549527.56279..."
3,4,Layton Downtown,Urban Center,2,Layton,None,"100,000",0.75 to 4.0,20 to 100,"Significant commerce. Regional ""destination"" r...",Intersection of major arterials and/or freeway...,Junction points of the high capacity transit n...,Yes,4 to 10,Amendment 4: Boundary Adjustment,4,"{""rings"": [[[419554.49820000026, 4545424.31719..."
4,5,Gentile & Fairfield Town Center,City Center,3,Layton,,"25,000",0.5 to 1.5,10 to 50,"Grocery store, + Neighborhood Center uses",Intersection of major collector with an arterial,Intersection of frequent bus lines --or-- serv...,Yes,2 to 5,Amendment 4: Boundary Adjustment,5,"{""rings"": [[[420429.2611999996, 4546442.0261],..."


In [17]:
policy_polys['DwellingPerAcre'].value_counts()

6 to 25      82
10 to 50     73
N/A          47
20 to 100    19
40 and up     2
Name: DwellingPerAcre, dtype: int64

In [18]:
policy_polys['NonResFAR'].value_counts()

0.35 to 1.0    82
0.5 to 1.5     73
Varies         34
0.75 to 4.0    19
N/A            13
1.0 and up      2
Name: NonResFAR, dtype: int64

In [19]:
policy_polys['Phase'].value_counts()

2    128
1     66
3     29
Name: Phase, dtype: int64

In [20]:
# create lookups
dua_lu = {"6 to 25":25, 
          "10 to 50": 50, 
          "20 to 100":100, 
          "40 and up": None,
          "N/A":None}

far_lu = {"0.35 to 1.0":1, 
          "0.5 to 1.5": 1.5, 
          "0.75 to 4.0":4,
          "N/A": None,
          "Varies":None,
          "1.0 and up":None}

phase_lu = {1:[2019,2030],
            2:[2031,2040],
            3:[2041,2050]}

In [21]:
# create functions to set lookups
def set_dua(in_value):
    return dua_lu[in_value]

def set_far(in_value):
    return far_lu[in_value]

def set_phase_begin_year(in_value):
    return phase_lu[in_value][0]

def set_phase_end_year(in_value):
    return phase_lu[in_value][1]

In [22]:
#  set values using lookups
policy_polys['max_dua_new'] = policy_polys['DwellingPerAcre'].apply(set_dua)
policy_polys['max_far_new'] = policy_polys['NonResFAR'].apply(set_far)
policy_polys['phase_begin_year'] = policy_polys['Phase'].apply(set_phase_begin_year)
policy_polys['phase_end_year'] = policy_polys['Phase'].apply(set_phase_end_year)

In [23]:
policy_polys['AreaType'].value_counts()

Neighborhood Center    82
City Center            73
Employment District    23
Urban Center           19
Industrial District    13
Special District       11
Metropolitan Center     2
Name: AreaType, dtype: int64

In [25]:
# use spatial join (average, mode) to get values from previous years
parcels_with_policy = r'.\Outputs\scratch.gdb\_00_parcels_with_policy'
parcels_with_policy_lyr = arcpy.MakeFeatureLayer_management(parcels_with_policy, 'parcels_with_policy_lyr') 
arcpy.SelectLayerByAttribute_management(parcels_with_policy_lyr, 'NEW_SELECTION', "locnote <> 'CBDStaging'")
parcels_with_policy_pts = arcpy.FeatureToPoint_management(parcels_with_policy_lyr, os.path.join(gdb, '_00_parcels_with_policy_pts'), "INSIDE")


In [26]:
# use spatial join to summarize max dua, max far, year, and type
target_features = new_polygons
join_features = parcels_with_policy_pts
output_features = os.path.join(gdb, "_01_centers_old_policy_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# max_dua
fieldindex = fieldmappings.findFieldMapIndex('max_dua')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# max far
fieldindex = fieldmappings.findFieldMapIndex('max_far')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mean'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# year
fieldindex = fieldmappings.findFieldMapIndex('year')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Min'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# building types allowed
fieldindex = fieldmappings.findFieldMapIndex('type1')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('type2')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('type3')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('type4')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('type5')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('type6')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('type7')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

fieldindex = fieldmappings.findFieldMapIndex('type8')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

In [27]:
sj_df = pd.DataFrame.spatial.from_featureclass(sj[0])
sj_df = sj_df[['id','max_dua','max_far','year','type1','type2','type3','type4','type5','type6','type7','type8', 'locnote']].copy()
sj_df['max_dua'] = sj_df['max_dua'].round(1)
sj_df['max_far'] = sj_df['max_far'].round(1)
sj_df.head(5)

,id,max_dua,max_far,year,type1,type2,type3,type4,type5,type6,type7,type8,locnote
0,1,10.0,0.8,2015.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,WFRCCenters
1,2,4.6,0.6,2015.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,WFRCCenters
2,3,29.7,2.0,2015.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,WFRCCenters
3,4,29.6,1.9,2015.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,WFRCCenters
4,5,10.0,0.8,2037.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,WFRCCenters


In [28]:
joined = policy_polys.merge(sj_df, left_on='id', right_on='id', how='left')
joined.head(5)

,OBJECTID,AreaName,AreaType,Phase,CityPrimary,CityOther,CatchmentPop,NonResFAR,DwellingPerAcre,IndicatorLU,MinRoadway,Transit,Walkable,Stories,Notes,id,SHAPE,max_dua_new,max_far_new,phase_begin_year,phase_end_year,max_dua,max_far,year,type1,type2,type3,type4,type5,type6,type7,type8,locnote
0,1,West Layton Town Center,City Center,2,Layton,None,"25,000",0.5 to 1.5,10 to 50,"Grocery store, + Neighborhood Center uses",Intersection of major collector with an arterial,Intersection of frequent bus lines --or-- serv...,Yes,2 to 5,"Amendment 4: Boundary Adjustment, Type Change",1,"{'rings': [[[415395.8512000004, 4546928.454], ...",50.0,1.5,2031,2040,10.0,0.8,2015.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,WFRCCenters
1,2,Layton Industrial Center,Industrial District,2,Layton,None,None,N/A,N/A,"Light and heavy industry, warehousing, inciden...",Freight oriented,Bus service,No,None,Amendment 4: Boundary Adjustment,2,"{'rings': [[[416374.0889999997, 4547760.0077],...",NaN,NaN,2031,2040,4.6,0.6,2015.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,WFRCCenters
2,3,Layton Midtown,Urban Center,1,Layton,Clearfield,"100,000",0.75 to 4.0,20 to 100,"Significant commerce. Regional ""destination"" r...",Intersection of major arterials and/or freeway...,Junction points of the high capacity transit n...,Yes,4 to 10,Amendment 4: Boundary Adjustment,3,"{'rings': [[[417032.33100000024, 4549527.56279...",100.0,4.0,2019,2030,29.7,2.0,2015.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,WFRCCenters
3,4,Layton Downtown,Urban Center,2,Layton,None,"100,000",0.75 to 4.0,20 to 100,"Significant commerce. Regional ""destination"" r...",Intersection of major arterials and/or freeway...,Junction points of the high capacity transit n...,Yes,4 to 10,Amendment 4: Boundary Adjustment,4,"{'rings': [[[419554.49820000026, 4545424.31719...",100.0,4.0,2031,2040,29.6,1.9,2015.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,WFRCCenters
4,5,Gentile & Fairfield Town Center,City Center,3,Layton,,"25,000",0.5 to 1.5,10 to 50,"Grocery store, + Neighborhood Center uses",Intersection of major collector with an arterial,Intersection of frequent bus lines --or-- serv...,Yes,2 to 5,Amendment 4: Boundary Adjustment,5,"{'rings': [[[420429.2611999996, 4546442.0261],...",50.0,1.5,2041,2050,10.0,0.8,2037.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,WFRCCenters


In [29]:
# set other attributes
# policy_polys['year'] = None
# policy_polys['locnote'] = 'WFRCCenters'
joined['mponote'] = 'WFRC'

# joined['type1'] = joined['type1']
# joined['type2'] = joined['type2']
# joined['type3'] = joined['type3']
# joined['type4'] = joined['type4']
# joined['type5'] = joined['type5'].astype(int)
# joined['type6'] = joined['type6'].astype(int)
# joined['type7'] = joined['type7'].astype(int)
# joined['type8'] = joined['type8'].astype(int)

# cast type columns as int
joined['type1'] = joined['type1'].fillna(0).astype(int)
joined['type2'] = joined['type2'].fillna(0).astype(int)
joined['type3'] = joined['type3'].fillna(0).astype(int)
joined['type4'] = joined['type4'].fillna(0).astype(int)
joined['type5'] = joined['type5'].fillna(0).astype(int)
joined['type6'] = joined['type6'].fillna(0).astype(int)
joined['type7'] = joined['type7'].fillna(0).astype(int)
joined['type8'] = joined['type8'].fillna(0).astype(int)


                            
joined['phase_count'] = 1
joined['phase_years'] = ""
joined['phase_maxduas'] = ""
joined['phase_maxfars'] = ""
joined['phase_types'] = ""

In [30]:
# industrial district
joined.loc[joined['AreaType']=='Industrial District', 'max_dua'] = .4
joined.loc[joined['AreaType']=='Industrial District', 'type3'] = 1

# metropolitan center
joined.loc[joined['AreaType']=='Metropolitan Center', 'max_dua'] = 65
joined.loc[joined['AreaType']=='Metropolitan Center', 'max_far'] = 0

joined.loc[joined['AreaType']=='Metropolitan Center', 'type1'] = 1
joined.loc[joined['AreaType']=='Metropolitan Center', 'type2'] = 1
joined.loc[joined['AreaType']=='Metropolitan Center', 'type3'] = 0
joined.loc[joined['AreaType']=='Metropolitan Center', 'type4'] = 1
joined.loc[joined['AreaType']=='Metropolitan Center', 'type5'] = 1
joined.loc[joined['AreaType']=='Metropolitan Center', 'type6'] = 0
joined.loc[joined['AreaType']=='Metropolitan Center', 'type7'] = 0
joined.loc[joined['AreaType']=='Metropolitan Center', 'type8'] = 0

joined.loc[joined['AreaType']=='Metropolitan Center', 'phase_count'] = 3
joined.loc[joined['AreaType']=='Metropolitan Center', 'phase_years'] = "2019,2025,2035"
joined.loc[joined['AreaType']=='Metropolitan Center', 'phase_maxduas'] = "65,95,125"
joined.loc[joined['AreaType']=='Metropolitan Center', 'phase_maxfars'] = "9,10.5,12"
joined.loc[joined['AreaType']=='Metropolitan Center', 'phase_types'] = "11011000,11011000,11011000"
joined.loc[joined['AreaType']=='Metropolitan Center', 'locnote'] = "CBDStaging"

joined.loc[joined['year'] < 2019 , 'year'] = 2019

# under certain conditions, set max far and dua with newer values
joined.loc[((joined['max_dua'].isna() == True) | (joined['max_dua'] == 0)) & (joined['max_dua_new'] > 0), 'max_dua'] = joined['max_dua_new']
joined.loc[((joined['max_far'].isna() == True) | (joined['max_far'] == 0)) & (joined['max_far_new'] > 0), 'max_far'] = joined['max_far_new']

# round max dua values above 6
joined.loc[joined['max_dua'] > 6, 'max_dua'] = joined['max_dua'].round(0)

# create composite type key
joined['type_composite'] = (joined['type1'].astype(str) + joined['type2'].astype(str) + joined['type3'].astype(str) + joined['type4'].astype(str) + 
                            joined['type5'].astype(str) + joined['type6'].astype(str) + joined['type7'].astype(str) + joined['type8'].astype(str))

In [77]:
joined.head()

,OBJECTID,AreaName,AreaType,Phase,CatchmentPop,NonResFAR,DwellingPerAcre,IndicatorLU,MinRoadway,Transit,Walkable,Stories,id,SHAPE,max_dua_new,max_far_new,phase_begin_year,phase_end_year,max_dua,max_far,year,type1,type2,type3,type4,type5,type6,type7,type8,locnote,mponote,phase_count,phase_years,phase_maxduas,phase_maxfars,phase_types,type_composite
0,1,Farmington Downtown,Neighborhood Center,3,"2,500",0.35 to 1.0,6 to 25,"Retail (pharmacy, restaurant) and/or civic (el...",Intersection of collectors / along a major col...,Frequent bus service (or better),Yes,1 to 3,1,"{'rings': [[[425020.9299999997, 4536963.539999...",25.0,1.0,2041,2050,10.0,0.8,2037.0,1,1,0,1,1,0,0,0,WFRCCenters,WFRC,1,,,,,11011000
1,2,Station Park/North Station,Urban Center,1,"100,000",0.75 to 4.0,20 to 100,"Significant commerce. Regional ""destination"" r...",Intersection of major arterials and/or freeway...,Junction points of the high capacity transit n...,Yes,4 to 10,2,"{'curveRings': [[[422130.9299999997, 4539255.9...",100.0,4.0,2019,2030,30.0,2.0,2019.0,0,1,0,1,1,0,0,0,WFRCCenters,WFRC,1,,,,,01011000
2,3,Centerville Main Street,Neighborhood Center,3,"2,500",0.35 to 1.0,6 to 25,"Retail (pharmacy, restaurant) and/or civic (el...",Intersection of collectors / along a major col...,Frequent bus service (or better),Yes,1 to 3,3,"{'curveRings': [[[426023.38999999966, 4529335....",25.0,1.0,2041,2050,20.0,0.8,2037.0,0,1,0,1,1,0,0,0,WFRCCenters,WFRC,1,,,,,01011000
3,4,North Salt Lake Town Center,City Center,2,"25,000",0.5 to 1.5,10 to 50,"Grocery store, + Neighborhood Center uses",Intersection of major collector with an arterial,Intersection of frequent bus lines --or-- serv...,Yes,2 to 5,4,"{'curveRings': [[[423047.6200000001, 4520431.4...",50.0,1.5,2031,2040,25.0,1.0,2019.0,0,1,0,1,1,0,0,0,WFRCCenters,WFRC,1,,,,,01011000
4,5,University of Utah,Special District,1,None,Varies,N/A,Regionally significant special purpose and non...,Varies,Varies,Varies,Varies,5,"{'rings': [[[429590.76999999955, 4514131.05000...",NaN,NaN,2019,2030,NaN,NaN,NaN,0,0,0,0,0,0,0,0,None,WFRC,1,,,,,00000000


In [31]:
policy_polys_export = joined[['id','max_dua', 'max_dua_new', 'max_far','max_far_new', 'year', 'type1', 'type2', 'type3', 'type4','type5', 'type6', 'type7', 'type8', 'type_composite','locnote', 'mponote',
                                    'AreaName', 'AreaType', 'phase_begin_year', 'phase_end_year', 'phase_count','phase_years', 'phase_maxduas', 'phase_maxfars','phase_types','SHAPE']].copy()

policy_polys_export.head()

,id,max_dua,max_dua_new,max_far,max_far_new,year,type1,type2,type3,type4,type5,type6,type7,type8,type_composite,locnote,mponote,AreaName,AreaType,phase_begin_year,phase_end_year,phase_count,phase_years,phase_maxduas,phase_maxfars,phase_types,SHAPE
0,1,10.0,50.0,0.8,1.5,2019.0,1,1,0,1,1,0,0,0,11011000,WFRCCenters,WFRC,West Layton Town Center,City Center,2031,2040,1,,,,,"{'rings': [[[415395.8512000004, 4546928.454], ..."
1,2,0.4,NaN,0.6,NaN,2019.0,0,1,1,1,1,0,0,0,01111000,WFRCCenters,WFRC,Layton Industrial Center,Industrial District,2031,2040,1,,,,,"{'rings': [[[416374.0889999997, 4547760.0077],..."
2,3,30.0,100.0,2.0,4.0,2019.0,0,1,1,1,1,0,0,0,01111000,WFRCCenters,WFRC,Layton Midtown,Urban Center,2019,2030,1,,,,,"{'rings': [[[417032.33100000024, 4549527.56279..."
3,4,30.0,100.0,1.9,4.0,2019.0,0,1,0,1,1,0,0,0,01011000,WFRCCenters,WFRC,Layton Downtown,Urban Center,2031,2040,1,,,,,"{'rings': [[[419554.49820000026, 4545424.31719..."
4,5,10.0,50.0,0.8,1.5,2037.0,1,1,0,1,1,0,0,0,11011000,WFRCCenters,WFRC,Gentile & Fairfield Town Center,City Center,2041,2050,1,,,,,"{'rings': [[[420429.2611999996, 4546442.0261],..."


In [32]:
policy_polys_export.spatial.to_featureclass(location=os.path.join(gdb2, 'policy_override_polygons_2019'),sanitize_columns=False)

'e:\\Projects\\REMM-Input-Data-Prep-2019\\Policy-Layer\\Outputs\\policy_override.gdb\\policy_override_polygons_2019'

In [80]:
# # import subprocess
# os.startfile(r".\Policy_Override_Review.aprx")

# REVIEW POLYGON OUTPUT BEFORE PROCEEDING TO NEXT SCRIPT!